Declarations - always run this before proceeding to diff sections below

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import serial

# Variables
heart_amplitude = 0.5
heart_frequency = 40 / 60
lung_amplitude = 0.5
lung_frequency = 10 / 60
sampling_rate = 20
sampling_interval = 1 / sampling_rate
lead_screw_lead = 8
steps_per_revolution = 200
wave_period = 1 / lung_frequency

t_fine = np.arange(0, wave_period, 0.01)
t_sampled = np.arange(0, wave_period, sampling_interval)

def generate_motion(t, heart_amplitude, heart_frequency, lung_amplitude, lung_frequency):
    heart_motion = heart_amplitude * np.sin(2 * np.pi * heart_frequency * t)
    lung_motion = lung_amplitude * np.sin(2 * np.pi * lung_frequency * t)
    combined_motion = heart_motion + lung_motion
    return heart_motion, lung_motion, combined_motion

def encode_motor_instructions(steps, sampling_rate):
    data = ['P', f'{sampling_rate:02X}']  # Start byte 'P' and sampling rate represented in 2 hex bytes
    for step in steps:
        sign = '1' if step >= 0 else '0'
        magnitude = f'{abs(step):02X}'    # mag in 2 hex bytes, sign in 1 byte, total 3 bytes per datapoint
        data.append(sign)                
        data.append(magnitude)
    data.append('V')                      # End byte 'V'
    return ''.join(data).encode('utf-8')

In [9]:
# Sampling
heart_motion_sampled, lung_motion_sampled, combined_motion_sampled = generate_motion(t_sampled, heart_amplitude, heart_frequency, lung_amplitude, lung_frequency)
combined_motion_sampled_mm = combined_motion_sampled * 10 # in mm

# Differences between consecutive positions
displacements_mm = np.diff(combined_motion_sampled_mm)
steps_per_mm = steps_per_revolution / lead_screw_lead 
motor_steps = np.round(displacements_mm * steps_per_mm).astype(int)

motor_instructions = encode_motor_instructions(motor_steps, sampling_rate)

print(f'Non-encoded Motor Steps: {motor_steps}')
print(f'Encoded Motor Instructions: {motor_instructions}')


Non-encoded Motor Steps: [ 33  31  29  26  22  17  12   6   0  -5 -10 -14 -17 -20 -21 -21 -21 -19
 -16 -12  -8  -3   3   8  12  17  21  23  25  26  26  24  22  18  14   9
   3  -3  -8 -14 -19 -23 -27 -29 -30 -31 -30 -28 -25 -21 -16 -11  -6  -1
   4   9  13  16  18  19  19  18  16  13   9   4  -1  -6 -11 -16 -21 -25
 -28 -30 -31 -30 -29 -27 -23 -19 -14  -8  -3   3   9  14  18  22  24  26
  26  25  23  21  17  12   8   3  -3  -8 -12 -16 -19 -21 -21 -21 -20 -17
 -14 -10  -5   0   6  12  17  22  26  29  31]
Encoded Motor Instructions: b'P1412111F11D11A11611110C10610000500A00E01101401501501501301000C00800310310810C11111511711911A11A11811611210E10910300300800E01301701B01D01E01F01E01C01901501000B00600110410910D11011211311311211010D10910400100600B01001501901C01E01F01E01D01B01701300E00800310310910E11211611811A11A11911711511110C10810300300800C01001301501501501401100E00A00510010610C11111611A11D11FV'


In [10]:
ser = serial.Serial('COM4', 9600, timeout=5)
time.sleep(2) 
ser.flush()
ser.flushInput()
ser.write(motor_instructions)

while True:
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        if line == "Y":
            break
        print(line)

# Close serial connection
ser.close()
print("End")

50
End
